In [ ]:
import pandas as pd
from io import StringIO
raw = StringIO("""producto,precio,cantidad,categoria
Manzana,100,3,Fruta
Banana,50,5,Fruta
Pera,80,2,Fruta
Notebook,350000,1,Electrónica
Mouse,8000,2,Electrónica
Manzana,100,,Fruta
Banana,50,3,fruta
Camisa,25000,1, Indumentaria
Camisa ,25000,2,Indumentaria
Mouse,8000,1,Electrónica
""")
df_raw = pd.read_csv(raw)
df_raw.to_csv("ventas.csv", index=False)
print("Archivo creado: ventas.csv")
df_raw


In [ ]:
import pandas as pd

df = pd.read_csv("ventas.csv")
display(df.head())
print("\nInfo:")
print(df.info())
print("\nShape:", df.shape)


In [ ]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)
df.head()


In [ ]:
# strip en columnas de texto típicas
for col in ["producto", "categoria"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# (Opcional) normalización simple de categoría
if "categoria" in df.columns:
    df["categoria"] = df["categoria"].str.title()

# Duplicados exactos (opcional según dataset)
dup_count = df.duplicated().sum()
print("Duplicados exactos:", dup_count)
# df = df.drop_duplicates()

df.head(10)


In [ ]:
# Intento de casteo seguro
if "precio" in df.columns:
    df["precio"] = pd.to_numeric(df["precio"], errors="coerce")
if "cantidad" in df.columns:
    df["cantidad"] = pd.to_numeric(df["cantidad"], errors="coerce")

print("Nulos por columna antes:")
print(df.isna().sum())

# Estrategia propuesta (ajustá en vivo según sugieran):
# - Si falta cantidad, la descarto (no puedo inventar unidades).
# - Si falta categoría, la marco como "Sin categoría".
# - Si falta precio y no hay manera de imputar, la descarto.

subset_drop = []
if "cantidad" in df.columns: subset_drop.append("cantidad")
if "precio"   in df.columns: subset_drop.append("precio")

df = df.dropna(subset=subset_drop)

if "categoria" in df.columns:
    df["categoria"] = df["categoria"].fillna("Sin Categoría")

print("\nNulos por columna después:")
print(df.isna().sum())
df.head()


In [ ]:
resumen_producto = (
    df.groupby("producto", dropna=False)["ingresos"]
      .sum()
      .reset_index()
      .sort_values("ingresos", ascending=False)
)

print("Top 10 productos por ingresos:")
resumen_producto.head(10)


In [ ]:
if "categoria" in df.columns:
    resumen_categoria = (
        df.groupby("categoria", dropna=False)["ingresos"]
          .sum()
          .reset_index()
          .sort_values("ingresos", ascending=False)
    )
    print("Ingresos por categoría:")
    resumen_categoria


In [ ]:
resumen_producto.to_csv("ventas_limpias.csv", index=False)
print("OK -> ventas_limpias.csv")


In [ ]:
from datetime import datetime
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
resumen_producto.to_csv(f"ventas_limpias_{ts}.csv", index=False)
print(f"OK -> ventas_limpias_{ts}.csv")
